In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.1.0 (SDL 2.0.16, Python 3.8.8)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
mixer.init()

alert = mixer.Sound('alarm.wav')
#alert.play()
#time.sleep(0.1)
#alert.play() 
#/Users/rana/opt/anaconda3/lib/python3.8/site-packages/cv2/data//haarcascade_frontalface_default.xml

In [3]:
#face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

face =cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_righteye_2splits.xml')


In [4]:
lbl=['Sleepy','Active']

In [5]:
model = load_model('models/cnncat1.h5')
path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
count=0
score=0
thicc=2
rpred=[99]
lpred=[99]



In [6]:

while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2] 

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye =  reye.detectMultiScale(gray)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (100,100,100) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye=  r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        
        # predict probabilities for test set
        #rpred_probs = model.predict(r_eye, verbose=0)
        # predict crisp classes for test set
        #rpred_classes = model.predict_classes(r_eye, verbose=0)
        #rpred=(model.predict(r_eye) > 0.5).astype("int32")
        #rpred =model.predict>0.5.astype('int32')
        #rpred = model.predict_classes(r_eye)
        #rpred=model.predict(r_eye)
        #rpred = np.round(rpred).astype(int)
        #classes_x=np.argmax(rpred,axis=1)
        rpred = np.argmax(model.predict(r_eye), axis=-1)
        if(rpred[0]==1):
            lbl='Active' 
        if(rpred[0]==0):
            lbl='Sleepy'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)  
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        
        #lpred_probs = model.predict(l_eye, verbose=0)
        #lpred_classes = model.predict_classes(l_eye, verbose=0)
        #lpred =model.predict>0.5.astype('int32')
        #lpred=(model.predict(l_eye) > 0.5).astype("int32")
        #lpred = model.predict_classes(l_eye)
        lpred = np.argmax(model.predict(l_eye), axis=-1)
        #lpred=model.predict(l_eye) 
        #lpred = np.round(lpred).astype(int)
        
        if(lpred[0]==1):
            lbl='Active'   
        if(lpred[0]==0):
            lbl='Sleepy'
        break
        

    if(rpred[0]==0 and lpred[0]==0):
        score=score+1
        cv2.putText(frame,"Sleepy",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    #if(rpred[0]==1 or lpred[0]==1):
    else:
        score=score-1
        cv2.putText(frame,"Active",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
        
    if(score<0):
        score=0   
        cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    if(score>15):
        #person is feeling sleepy so we beep the alarm
        cv2.imwrite(os.path.join(path,'image.jpg'),frame)
        try:
            cv2.putText(frame, "Drowsiness Alert!!!", (100, height-20), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)

            alert.play()
            
        except:  # isplaying = False
            pass
        if(thicc<16):
            thicc= thicc+2
        else:
            thicc=thicc-2
            if(thicc<2):
                thicc=2
        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),thicc) 
    cv2.imshow('frame',frame)
    time.sleep(0.3) # Sleep for 3 seconds
    if cv2.waitKey(1) &  0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
#vs.stop()


KeyboardInterrupt: 